In [1]:
from datasets import load_dataset
import pandas as pd
dataset = load_dataset("mteb/tweet_sentiment_extraction")
df = pd.DataFrame(dataset['train'])


/root/miniconda3/envs/my-env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from transformers import BertTokenizer
from datasets import load_dataset

# データセットの読み込み
dataset = load_dataset("mteb/tweet_sentiment_extraction")

# BERTトークナイザーの初期化
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# パディングトークンの追加
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# トークナイズ関数
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

# トークナイズの適用
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map: 100%|██████████| 3534/3534 [00:01<00:00, 3349.77 examples/s]


In [4]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

In [5]:
from transformers import BertForSequenceClassification

# BERTモデルの初期化
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
import evaluate
import numpy as np
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   return metric.compute(predictions=predictions, references=labels)

In [7]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
   output_dir="test_trainer",
   per_device_train_batch_size=1,  # Reduce batch size here
   per_device_eval_batch_size=1,    # Optionally, reduce for evaluation as well
   gradient_accumulation_steps=4
   )


trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=small_train_dataset,
   eval_dataset=small_eval_dataset,
   compute_metrics=compute_metrics,

)

trainer.train()


Step,Training Loss
500,0.697300


TrainOutput(global_step=750, training_loss=0.5472848002115885, metrics={'train_runtime': 113.5092, 'train_samples_per_second': 26.43, 'train_steps_per_second': 6.607, 'total_flos': 789340253184000.0, 'train_loss': 0.5472848002115885, 'epoch': 3.0})